In [ ]:
%matplotlib inline
#(env)$ python3 -m ipykernel install --user --name=env
from fgspectra import cross as fgc
from fgspectra import power as fgp
from fgspectra import frequency as fgf
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import sensitivity_calculator as sens
#from scipy.optimize import minimize
#import sys
#sys.path.append('/root/')
#import pcl_actpol_utility_v4 as pau
import emcee
import corner
import tqdm
import os
def get_hivon_fsky(mask):
    npix = len(mask)
    non0 = np.where(mask!=0)[0]
    fs = len(non0)/float(npix)
    w2 = np.sum(mask[non0]**2)/(npix*fs)
    w4 = np.sum(mask[non0]**4)/(npix*fs)
    return fs*w2**2/w4
#range for power spectra
ells = np.arange(2, 8000)
#parameters for fgspectra
par = {
    "nu_0": 150.0,
    "ell_0": 3000,
    "T_CMB": 2.725,
    "T_d": 9.7,
    "a_tSZ": 3.30,
    "a_kSZ": 1.60,
    "a_p": 6.90,
    "beta_p": 2.20,
    "a_c": 4.90,
    "beta_c": 2.20,
    "n_CIBC": 1.20,
    "xi": 0.1,
    "a_s": 3.10,
    "a_g": 2.80,
}
# define the models from fgspectra
ksz = fgc.FactorizedCrossSpectrum(fgf.ConstantSED(), fgp.kSZ_bat())
cibp = fgc.FactorizedCrossSpectrum(fgf.ModifiedBlackBody(), fgp.PowerLaw())
radio = fgc.FactorizedCrossSpectrum(fgf.PowerLaw(), fgp.PowerLaw())
cirrus = fgc.FactorizedCrossSpectrum(fgf.PowerLaw(), fgp.PowerLaw())
# if there are correlations between components,
# have to define them in a joined spectrum
tSZ_and_CIB = fgc.CorrelatedFactorizedCrossSpectrum(
    fgf.Join(fgf.ThermalSZ(), fgf.CIB()), fgp.SZxCIB_Addison2012()
)
# for testing purposes we'll also compute the tSZ and clustered CIB alone
tsz = fgc.FactorizedCrossSpectrum(fgf.ThermalSZ(), fgp.tSZ_150_bat())
cibc = fgc.FactorizedCrossSpectrum(fgf.CIB(), fgp.PowerLaw())
def get_fg_model(freqs, ells, par):
    return (
        par["a_kSZ"] * ksz({"nu": freqs}, {"ell": ells, "ell_0": par["ell_0"]}),
        par["a_p"]
        * cibp(
            {
                "nu": freqs,
                "nu_0": par["nu_0"],
                "temp": par["T_d"],
                "beta": par["beta_p"],
            },
            {"ell": ells, "ell_0": par["ell_0"], "alpha": 2},
        ),
        # joint tSZ, CIB, and their correlation
        tSZ_and_CIB(
            {
                "kwseq": (
                    {"nu": freqs, "nu_0": par["nu_0"]},
                    {
                        "nu": freqs,
                        "nu_0": par["nu_0"],
                        "temp": par["T_d"],
                        "beta": par["beta_c"],
                    },
                )
            },
            {
                "kwseq": (
                    {"ell": ells, "ell_0": par["ell_0"], "amp": par["a_tSZ"]},
                    {
                        "ell": ells,
                        "ell_0": par["ell_0"],
                        "alpha": 2 - par["n_CIBC"],
                        "amp": par["a_c"],
                    },
                    {
                        "ell": ells,
                        "ell_0": par["ell_0"],
                        "amp": -par["xi"] * np.sqrt(par["a_tSZ"] * par["a_c"]),
                    },
                )
            },
        ),
        par["a_s"]
        * radio(
            {"nu": freqs, "nu_0": par["nu_0"], "beta": -0.5 - 2},
            {"ell": ells, "ell_0": par["ell_0"], "alpha": 2},
        ),
        par["a_g"]
        * cirrus(
            {"nu": freqs, "nu_0": par["nu_0"], "beta": 3.8 - 2},
            {"ell": ells, "ell_0": par["ell_0"], "alpha": -0.7},
        ),
        # THESE TWO ARE REDUDUNDANT, BUT USEFUL FOR TESTING
        par["a_tSZ"]
        * tsz({"nu": freqs, "nu_0": par["nu_0"]}, {"ell": ells, "ell_0": par["ell_0"]}),
        par["a_c"]
        * cibc(
            {
                "nu": freqs,
                "nu_0": par["nu_0"],
                "temp": par["T_d"],
                "beta": par["beta_c"],
            },
            {"ell": ells, "ell_0": par["ell_0"], "alpha": 2 - par["n_CIBC"]},
        ),
    )

###############################################################################################333
#use sensitivity calculator 
inputs = {'diameter': 5.7, 't': 273, 'wfe': 10.7, 'eta': 0.98, 'doe': 0.8, 't_int': 1, 'pixelYield': 0.8, 
        'szCamNumPoln': 1, 'eorSpecNumPoln': 2, 't_filter_cold': np.array([0.593, 0.84 , 0.86, 0.86, 0.89 ]), 
        't_lens_cold': np.array([0.98, 0.98, 0.98, 0.98, 0.98]), 
        't_uhdpe_window': np.array([0.96, 0.97, 0.97, 0.97, 0.97]), 
        'singleModedAOmegaLambda2': np.array([1., 1., 1., 1., 1.]), 
        'spatialPixels': np.array([39762, 20808, 10368, 10368,  7938]), 
        'eqbw': np.array([9.7e+10, 3.0e+10, 3.5e+10, 6.0e+10, 5.6e+10]), 
        'centerFrequency': np.array([8.50e+11, 4.10e+11, 3.50e+11, 2.80e+11, 2.20e+11]), 'detectorNEP': 0, 
        'backgroundSubtractionDegradationFactor': 1, 'r': np.array([[100., 100., 100.],
                                            [100., 100., 100.],
                                            [100., 100., 100.],
                                            [100., 100., 100.],
                                            [100., 100., 100.]]), 
        'decimalPlaces': 3, 'observationElevationAngle': 45, 'outputFreq': True, 
        'detectorSpacing': np.array([1.45, 1.97, 2.75, 2.75, 3.13]), 'lyotStopAngle': 13.4}
# Alternatively, sens.getInputs("input.yaml")
#spillEfficiency = sens.getSpillEfficiency(inputs, oldFile=False)
#print(spillEfficiency)
spillEfficiency = sens.getSpillEfficiency(inputs, oldFile=True)
#print(spillEfficiency)
# spillEfficienc#y = np.array([0.8,0.5,0.7,0.95,0.5])
calculate = sens.calcByAngle(inputs["diameter"], inputs["t"], inputs["wfe"], inputs["eta"], inputs["doe"], inputs["t_int"], inputs["pixelYield"], inputs["szCamNumPoln"], inputs["eorSpecNumPoln"],
                            inputs["t_filter_cold"], inputs["t_lens_cold"], inputs["t_uhdpe_window"], spillEfficiency, inputs["singleModedAOmegaLambda2"],
                            inputs["spatialPixels"], inputs["eqbw"], inputs["centerFrequency"], inputs["detectorNEP"],
                            inputs["backgroundSubtractionDegradationFactor"], inputs["r"])
outputs = calculate(45) #45 degree angle
#get noise from sens, by l, temp noise, and pol noise
ell, N_ell_T_full, N_ell_P_full = sens.getNoiseCurves(inputs, outputs)
dl = ell*(ell+1)/(2*np.pi)
#get sky fraction fsky
NSIDE = 1024                        #mask_share/ccat_uniform_coverage_nside1024_201021.fits
hitmap_path = os.path.expanduser(f"mask_share/ccat_uniform_coverage_nside{NSIDE}_201021.fits")
mask_path = os.path.expanduser(f"mask_share/nside{NSIDE}_lat_cut_b10_230619.fits")
hitm = hp.read_map(hitmap_path)
maskm = hp.read_map(mask_path)
#hp.mollview(hitm*maskm)
#hp.mollview(maskm)
fsky = np.sum(hitm*maskm)/len(hitm) 
fsky = get_hivon_fsky(hitm*maskm)
print('fsky=%.4f'%fsky)

In [ ]:
#SO noise curves
from so_models_v3 import SO_Noise_Calculator_Public_v3_1_1 as SO_noise
mode = 'baseline'
so_fsky = 0.4
lat = SO_noise.SOLatV3point1(mode,survey_years=5.,survey_efficiency = 0.2*0.85)
lat_lmax = 10000
so_freq_list = np.array([27, 39, 93, 145, 225, 280])

ell_LA, N_ell_LA_T_full,N_ell_LA_P_full = lat.get_noise_curves(
        so_fsky, lat_lmax, 1, full_covar=False, deconv_beam=True)
'''
for curves, labels, title in zip([N_ell_LA_T_full], [so_freq_list], ["SO Noise Curves (T)"]):
    for curve, label in zip(curves, labels):
        plt.plot(ell_LA, curve, label=str(int(label))+' GHz')
    plt.yscale('log')
    plt.xscale('log')
    plt.xlim(10**2, 10**4)
    plt.ylim(10**-5, 10**3)
    plt.xlabel('$\ell$',fontsize=18)
    plt.ylabel('$N_\ell [\mu K^2]$',fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.title(title)
    plt.legend(loc='upper right')
    plt.grid()
    plt.show()

ccat_freq_list = np.array([8.50e+11, 4.05e+11, 3.48e+11, 2.80e+11, 2.22e+11])

for curves, labels, title in zip([N_ell_T_full], [ccat_freq_list], ["CCAT Noise Curves (T)"]):
    for curve, label in zip(curves, labels):
        plt.plot(ell, curve, label=str(int(label/1e9))+' GHz')
    plt.yscale('log')
    plt.xscale('log')
    plt.xlim(10**2, 10**4)
    plt.ylim(10**-5, 10**3)
    plt.xlabel('$\ell$',fontsize=18)
    plt.ylabel('$N_\ell [\mu K^2]$',fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.title(title)
    plt.legend(loc='upper right')
    plt.grid()
    plt.show()
'''

In [ ]:
#FTS resolution generator
def fts_res(centers):
    f=1000.000
    del_f =1.000
    ratio = del_f/f

    res = ratio * centers

    #print(res)
    return np.round(res, 4)

fts_res(1000)

In [ ]:
#Build signal and noise spectra
ccat_freq_list = np.array([850, 410, 350, 280, 220])
so_freq_list = np.array([27, 39, 93, 145, 225, 280])
#band centers to consider
#centerfreqs_full = np.concatenate((ccat_freq_list,so_freq_list))
#print(centerfreqs_full)
#frequency indices from sensitivity_calc
#[8.50e+11, 4.05e+11, 3.48e+11, 2.80e+11, 2.22e+11]
#    0          1        2         3         4
#frequency indices from SO freq list
#[25.7, 38.9, 92, 147.5, 225.7, 285.4]
#   0     1    2    3      4     5
#######################################s
ccat_center_indices = [1,2,3,4]
so_center_indices = []#[2,3,4,5]
#######################################
N_l_ccat = np.array([N_ell_T_full[i] for i in ccat_center_indices]) #noise curves from sens/CCAT
N_l_so = np.array([N_ell_LA_T_full[i] for i in so_center_indices]) #noise curves from SO

N_l = np.concatenate([arr for arr in [N_l_ccat, N_l_so] if len(arr) > 0])
print(f"N_l: {N_l}")
###########################################################################
ccat_freqs = np.array([ccat_freq_list[i] for i in ccat_center_indices])
so_freqs = np.array([so_freq_list[i] for i in so_center_indices])
centerfreqs = np.concatenate((ccat_freqs,so_freqs))
n_freq = len(centerfreqs)
print(centerfreqs)

#generate spectra as seen by the center frequencies
C_l_full = get_fg_model(centerfreqs, ells, par)
#C_l[foreground spectrum index][center frequency index1, center frequency index2]
#spectra ['a_kSZ','a_p','tSZ_CIB','a_s','a_g','a_tSZ','a_c'] #spectra
#           0       1       2       3     4      5      6
###########################################
fore_indices = [1,6] #[5,0,6,1,3]
############################################
C_l = [C_l_full[i] for i in fore_indices]
n_fore = len(C_l)
print(fore_indices)

#mismatch/offset frequencies
############################################
ccat_offset = fts_res(ccat_freqs)
so_offset = [] #[0.8,-1,1.5]
###########################################
offset = np.concatenate((ccat_offset,so_offset))
offset_centerfreqs = np.concatenate([(ccat_freqs + ccat_offset) if len(ccat_freqs) > 0 else [], (so_freqs + so_offset) if len(so_freqs) > 0 else []])
C_l_offset_full = get_fg_model(offset_centerfreqs, ells, par)
C_l_offset = [C_l_offset_full[i] for i in fore_indices]

'''
    "a_tSZ": 3.30,
    "a_kSZ": 1.60,
    "a_c": 4.90,
    "a_s": 3.10,
'''
#maps to get_fg_model  output
a_true_full = {
    0: 1.60,  # a_kSZ
    1: 6.90,  # a_p
    2: 0.10,  # xi_tSZxCIB
    3: 3.10,  # a_s
    4: 2.80,  # a_g
    5: 3.30,  # a_tSZ
    6: 4.90,  # a_c
}
'''
# a_true_full = {
#     0: 1.0,  # a_kSZ
#     1: 1.0,  # a_p
#     2: 1.0,  # tSZxCIB
#     3: 1.0,  # a_s
#     4: 1.0,  # a_g
#     5: 1.0,  # a_tSZ
#     6: 1.0,  # a_c
# }
'''
a_true = [a_true_full[i] for i in fore_indices]
#print(a_true)
#mapps to get_fg_model output
labels_full = [
    r"$a_{\mathrm{kSZ}}$", 
    r"$a_p$", 
    r"$a_{\mathrm{tSZ}\times\mathrm{CIB}}$", 
    r"$a_s$", #radio
    r"$a_g$", #dust
    r"$a_{\mathrm{tSZ}}$", 
    r"$a_c$"
]
labels = [labels_full[i] for i in fore_indices]
print(labels)

dcolors = [
    '#e41a1c',  # red
    '#4daf4a',  # green
    '#984ea3',  # purple
    '#ff7f00',  # orange
    '#a65628',  # brown
    '#f781bf',  # pink
    '#66c2a5',   # teal
    '#4b0082',  # indigo
    '#2e8b57',  # sea green
    '#800000',  # maroon
    '#3b3b98',  # dark indigo
    '#5d3fd3',  # dark violet
]

for f in range(n_freq):
    for s in range(n_fore):
        plt.plot(C_l[s][(f,f)], label=labels[s], color=dcolors[s])
        plt.plot(C_l_offset[s][(f,f)], label=f"{labels[s]} offset by {offset[f]} GHz", color=dcolors[s])
    plt.plot(N_l[f]*dl, label="D_l noise", color='black')
    plt.legend(loc = 'upper right')
    plt.yscale('log')
    plt.xlim(-500,9000)
    #plt.ylim(10e-3,10e3)
    plt.title(f"Foregrounds at {centerfreqs[f]}GHz")
    plt.gca().set_box_aspect(1)
    plt.show()


In [ ]:
#from pcl_actpol_utility
def read_binfile(path,lcut=100000):
   lbands = np.array(np.genfromtxt('%s'%path)[:,:2],dtype='int')
   if len(np.where(lbands[:,1]>lcut)[0]) != 0:
      lbands = lbands[:len(lbands)-len(np.where(lbands[:,1]>lcut)[0])]
   if lbands[0,0] == 0:
      lbands[0,0] = 2
   return lbands

def get_lbands_all(lbands):
   nbins = len(lbands)
   lmax = lbands[-1, -1]
   ell = np.arange(lmax + 1)
   dl = ell*(ell+1)/(2*np.pi)

   band = np.zeros(nbins)
   ell_bin = np.zeros(nbins)

   for i in range(nbins):
      band[i] = (lbands[i,1]-lbands[i,0])/2.
      ell_bin[i] = (lbands[i,0]+lbands[i,1])/2.

   return (ell_bin, band, nbins, lmax, ell, dl)

lmax_cut = 8000
binfile = 'BIN_ACTPOL_50_4_SC_low_ell'
lbands = read_binfile(binfile, lcut=lmax_cut) #bin limits in l
(ell_bin, band, n_bins, lmax, elxx, dxx) = get_lbands_all(lbands)
#ell_bin = bin centers
#band = bandwidth/2

#eq 2.5 in steve paper https://arxiv.org/pdf/1509.05934
nu_b = (2*ell_bin+1)*(2*band)*fsky #################################################################
#plt.plot(nu_b, label="nu_b")

#g_fac = pau.get_analytic_var_cross(fsky, ell_bin, band)
#plt.plot( 1/g_fac**2, label="nu_b from g_fac")

N_b = {}

# Rebin noise
for f1 in range(n_freq):
    N_b[f1] = np.zeros(n_bins)
    for i in range(n_bins):
        lmin, lmax = lbands[i]
        lrange = np.arange(lmin, lmax)
        D_N_l = N_l * dl
        N_b[f1][i] = np.mean(D_N_l[f1][lrange])


def rebin_C(tC_l):
   tC_b = {}
   for s in range(n_fore):
      tC_b[s] = {}
      for f1 in range(n_freq):
         for f2 in range(n_freq):
               tC_b[s][f1, f2] = np.zeros(n_bins)
               for i in range(n_bins):
                  lmin, lmax = lbands[i]
                  lrange = np.arange(lmin, lmax)
                  ################################################################## a_true means divide out the true amplitude
                  tC_b[s][f1, f2][i] = np.mean(tC_l[s][f1, f2][lrange]) / a_true[s]
                  ################################################################################ remove to fit to 1 instead (update a true)
   return tC_b

C_b = rebin_C(C_l)
C_b_offset = rebin_C(C_l_offset)

for s in range(n_fore):
      plt.plot(C_b[s][(0,0)], label=labels[s], color=dcolors[s])
      plt.plot(C_b_offset[s][(0,0)], label=f"{labels[s]} offset by {offset[0]} GHz", color=dcolors[s])

#plt.plot(N_b[0], label='noise')
plt.legend()
plt.yscale('log')
plt.title(f"Rebinned Templates at {centerfreqs[0]} GHz")
plt.show()

In [ ]:
freq_pairs = []
pair_index = {}
k = 0
for f1 in range(n_freq):
    for f2 in range(f1, n_freq):
        freq_pairs.append((f1, f2))
        pair_index[(f1, f2)] = k
        pair_index[(f2, f1)] = k
        k += 1
n_pairs = len(freq_pairs)
print(n_pairs)

#######################################################################################
#function that automatically adds only auto spectra noise for a given frequency pair
def sig_noise(f1, f2):
    C_total = sum(C_b[s][f1, f2] for s in range(n_fore)) # sum C_b
    if f1 == f2: #only if this term has an auto spectra include noise
        C_total += N_b[f1]
    return C_total

######################################################################################
#returns correct-looking covmat
cov_matrix = np.zeros((n_bins, n_pairs, n_pairs))
for a in range(n_freq):
    for b in range(n_freq):
        for c in range(n_freq):
            for d in range(n_freq):
                i = pair_index[(a, b)]
                j = pair_index[(c, d)]
                c_ac = sig_noise(a, c)
                c_bd = sig_noise(b, d)
                c_ad = sig_noise(a, d)
                c_bc = sig_noise(b, c)
                for bin in range(n_bins):
                    cov_matrix[bin, i, j] = (1 / nu_b[bin]) * (c_ac[bin]*c_bd[bin] + c_ad[bin]*c_bc[bin])
                cov_matrix[:, j, i] = cov_matrix[:, i, j]

#form cov_matrix[bin, fpair_1, fpair_2]
block = [[np.zeros((n_bins, n_bins)) for j in range(n_pairs)] for i in range(n_pairs)]

# Fill only the diagonal of each (n_bins × n_bins) block from cov_matrix
for i in range(n_pairs):
    for j in range(n_pairs):
        for b in range(n_bins):
            block[i][j][b, b] = cov_matrix[b, i, j]
  
# blockshow = block[0][0]
# plt.imshow(np.log10(blockshow), cmap='viridis')#, vmin=vmin, vmax=vmax)
# plt.colorbar(label='log covariance')
# plt.title("Covariance Matrix First Block")
# plt.show()

#arrange blocks into full matrix
cov_total_full = np.zeros((n_bins * n_pairs, n_bins * n_pairs))
for i in range(n_pairs):
    for j in range(n_pairs):
        #location in full matrix
        row_start = i * n_bins
        row_end = (i + 1) * n_bins
        col_start = j * n_bins
        col_end = (j + 1) * n_bins

        cov_total_full[row_start:row_end, col_start:col_end] = block[i][j]

#####################################################################

plt.imshow(np.log10(cov_total_full), cmap='viridis')#, vmin=vmin, vmax=vmax)
plt.colorbar(label='log covariance')
plt.title("log covariance matrix")
plt.show()

print(f"cov shape {np.shape(cov_total_full)}")


# cov_total_full2 = np.diag(2*1/nu_b*(C_b[0][0,0] + N_b[0])**2)
                          
# plt.imshow(np.log10(blockshow-cov_total_full2), cmap='viridis')#, vmin=vmin, vmax=vmax)
# plt.colorbar(label='log covariance')
# plt.title("log covariance matrix")
# plt.show()
'''
plt.imshow((cov_total_full), cmap='viridis')#, vmin=vmin, vmax=vmax)
plt.colorbar(label='covariance')
plt.title("covariance matrix")
plt.show()

def cov2corr(cov):
    stddev = np.sqrt(np.diag(cov))
    corr = cov / np.outer(stddev, stddev)
    np.fill_diagonal(corr, 1.0)
    return corr

corr_matrix = np.corrcoef(cov_total_full)
plt.imshow(np.log10(corr_matrix), cmap='viridis')#, vmin=vmin, vmax=vmax)
plt.colorbar(label='log correlation')
plt.title("log correlation_matrix")
plt.show()
plt.imshow((corr_matrix), cmap='viridis')#, vmin=vmin, vmax=vmax)
plt.colorbar(label='correlation')
plt.title("correlation_matrix")
plt.show()
'''

In [ ]:
#for fit with band centers, extend to include band center frequencies as labels
labels_f = labels + [rf"$\Delta\nu^{{{freq:.0f}}}_{{\mathrm{{band}}}}$" for freq in centerfreqs]
print(labels_f)
print(a_true)
a_true_f = a_true + [freq for freq in centerfreqs]
print(a_true_f)

In [ ]:
from multiprocessing import Pool
###################################################### GAUSS MCMC ###############################################
# Invert covariance matrix
cov_inv = np.linalg.inv(cov_total_full)
sign, logdet = np.linalg.slogdet(cov_total_full)

# Generate mock data vector from fiducial model
signal = np.zeros((n_bins, n_pairs))
for s in range(n_fore):
    for i, (f1, f2) in enumerate(freq_pairs):
        signal[:, i] += a_true[s] * C_b[s][f1, f2]


def log_likelihoodg(theta, data, cov_inv, logdet, offsets):
    a_params = theta[:n_fore]
    dnu_params = theta[n_fore:]

    shifted_freqs = centerfreqs + offsets + dnu_params
    C_b_shifted = rebin_C([get_fg_model(shifted_freqs, ells, par)[i] for i in fore_indices])

    T_templates = {}
    for s in range(n_fore):
        T = np.zeros((n_bins, n_pairs))
        for idx, (f1, f2) in enumerate(freq_pairs):  
            T[:, idx] = C_b_shifted[s][f1, f2].copy()
        T_templates[s] = T.flatten()

    model = sum(a_s * T_templates[s] for s, a_s in enumerate(a_params))
    delta = data - model
    return -0.5 * (delta @ cov_inv @ delta)

def log_priorg(theta, mu_dnu, sigma_dnu):
    a_params = theta[:n_fore]
    dnu_params = theta[n_fore:]

    if np.any(a_params < 0) or np.any(a_params > 9.0):
        return -np.inf

    gauss_stuff = -0.5 * np.sum(((dnu_params - mu_dnu) / sigma_dnu) ** 2)
    return gauss_stuff

def log_probabilityg(theta, data, cov_inv, logdet, offsets, mu_dnu, sigma_dnu):
    lp = log_priorg(theta, mu_dnu, sigma_dnu)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihoodg(theta, data, cov_inv, logdet, offsets)

# Run MCMC
def run_emcee_gauss(offsets, mu_array=None, sigma_array=None, plot=True):
    if offsets is None:
        offsets = np.zeros(n_freq)  #offset
    if mu_array is None:
        mu_array = np.zeros(n_freq)  # Prior centered at 0
    if sigma_array is None:
        sigma_array = np.ones_like(mu_array)  # Prior width of 1 GHz

    mock_data = np.random.multivariate_normal(mean=signal.flatten(), cov=cov_total_full)

    ndim = n_fore + n_freq
    nwalkers = 40
    initial = np.concatenate([
        np.array(a_true) + 0.01 * np.random.randn(nwalkers, n_fore),
        mu_array + 0.01 * np.random.randn(nwalkers, n_freq)
    ], axis=1)

    with Pool(processes=8) as pool:
        sampler = emcee.EnsembleSampler(
            nwalkers, ndim, log_probabilityg,
            args=(mock_data, cov_inv, logdet, offsets, mu_array, sigma_array),
            pool=pool
        )

        sampler.run_mcmc(initial, 200, progress=True)  # MUST be inside the 'with' block

    samples = sampler.get_chain(discard=50, thin=1, flat=True)

    #print("Autocorrelation times:", sampler.get_autocorr_time())
    
    p_means = samples.mean(axis=0)
    p_stds = samples.std(axis=0)
    return samples, p_means, p_stds


In [ ]:
samples_fid = []
for _ in range(1):
    s, m, std = run_emcee_gauss(
        offsets=np.zeros(n_freq),
        mu_array=np.zeros(n_freq),
        sigma_array=np.ones(n_freq) / 1000,
        plot=False
    )
    samples_fid.append(s)
    
means_fid = np.mean(samples_fid, axis=0)
stds_fid = np.std(samples_fid, axis=0)

In [ ]:

figure = corner.corner(
    np.vstack(samples_fid),
    labels=labels_f,
    truths=a_true_f,
    label_kwargs={"fontsize": 14},
    show_titles=True,
    title_fmt=".3f",
    title_kwargs={"fontsize": 12},
    bins = 40
)
plt.suptitle(f"CCAT:{ccat_freqs}GHz", fontsize=16, y=1.02)

In [ ]:
samples_sys = []
for _ in range(1):
    s, m, std = run_emcee_gauss(
        offsets=offset,
        mu_array=np.zeros(n_freq),
        sigma_array=np.ones(n_freq) / 1000,
        plot=False
    )
    samples_sys.append(s)

means_sys = np.mean(samples_sys, axis=0)
stds_sys = np.std(samples_sys, axis=0)

In [ ]:
samples_gp = []
for _ in range(1):
    s, m, std = run_emcee_gauss(
        offsets=np.zeros(n_freq),
        mu_array=np.zeros(n_freq),
        sigma_array=np.ones(n_freq),
        plot=False
    )
    samples_gp.append(s)
    
means_gp = np.mean(samples_gp, axis=0)
stds_gp = np.std(samples_gp, axis=0)

In [ ]:

figure = corner.corner(
    np.vstack(samples_fid),
    labels=labels_f,
    truths=a_true_f,
    label_kwargs={"fontsize": 14},
    show_titles=True,
    title_fmt=".3f",
    title_kwargs={"fontsize": 12},
    color="C0",
    plot_datapoints=False,
    fill_contours=False,
    levels=(0.68, 0.95),
    alpha=0.5
)

# Overlay second plot (e.g., comparison)
corner.corner(
    np.vstack(samples_sys),
    labels=labels_f,
    truths=a_true_f,
    label_kwargs={"fontsize": 14},
    show_titles=False,  # avoid duplicated titles
    color="C1",
    plot_datapoints=False,
    fill_contours=False,
    levels=(0.68, 0.95),
    fig=figure
)
# Overlay second plot (e.g., comparison)
corner.corner(
    np.vstack(samples_gp),
    labels=labels_f,
    truths=a_true_f,
    label_kwargs={"fontsize": 14},
    show_titles=False,  # avoid duplicated titles
    color="C2",
    plot_datapoints=False,
    fill_contours=False,
    levels=(0.68, 0.95),
    fig=figure
)
handles = [
    Patch(facecolor='none', edgecolor='C0', label="Fiducial", alpha=0.5),
    Patch(facecolor='none', edgecolor='C1', label="Systematic", alpha=0.5)
    Patch(facecolor='none', edgecolor='C2', label="Gaussian Prior", alpha=0.5)
]
plt.legend(
    handles=handles,
    loc='upper center',
    bbox_to_anchor=(0.5, 2),#bbox_to_anchor=(0.5, 4),  # x, y (move y negative to shift down)
    ncol=1,
    fontsize=12,
    frameon=False
)
plt.suptitle(f"CCAT Centers:{centerfreqs.tolist()}GHz with Offset:{ccat_offset}GHz", fontsize=16, y=1.02)
plt.show()

In [ ]:

bias_fid = np.abs(means_sys - means_fid) / stds_fid
bias_gp = np.abs(means_sys - means_gp) / stds_gp
degradation_fid = stds_sys/stds_fid
degradation_gp = stds_sys/stds_gp

param_names = labels
syslabels = ["Fiducial", "Systematic Offset", "1GHz Gaussian Prior"]
x = np.arange(len(syslabels))

fig, axes = plt.subplots(n_fore,1,figsize=(12, 12), sharex=True)
axes = axes.flatten()


for i in range(len(labels)):
    ax = axes[i]

    y_vals = [means_fid[i], means_sys[i], means_gp[i]]
    y_errs = [stds_fid[i], stds_sys[i], stds_gp[i]]

    # Plot error bars
    ax.errorbar(x, y_vals, yerr=y_errs, fmt='o', capsize=6, color='black')

    # Fiducial horizontal line and shaded uncertainty band
    ax.axhline(means_fid[i], color='gray', linestyle='dashed', label='Fiducial')
    ax.fill_between([-0.5, 2.5],
                    means_fid[i] - stds_fid[i],
                    means_fid[i] + stds_fid[i],
                    color='gray', alpha=0.2)

    
    # Annotate bias
    ax.text(0.5, 1.01, f"CCAT: {bias_fid[i]:.1f}σ bias (fid), {bias_gp[i]:.1f}σ bias (1GHz Gaussian), {degradation_fid[i]:.1f} degradation (fid), {degradation_gp[i]:.1f} degradation (1GHz Gaussian),",
        transform=ax.transAxes,
        ha='center', va='bottom', fontsize=12, color='black')
    
    #ax.text(0.1, 1.01,"SO",ha='center', va='bottom', fontsize=12, color='black')

    ax.set_xticks(x)
    ax.set_xticklabels(syslabels)
    ax.set_ylabel(param_names[i])
    ax.set_xlim(-0.5, 2.5)
    #ax.legend()

#fig.suptitle("Parameter Comparison: Fiducial vs Systematic", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()